In [ ]:
import random
import gzip
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import f1_score
import pickle
import math

In [ ]:
def reading_in(path):
    thing=[]
    for line in gzip.open(path):
        review_data = json.loads(line)
        subthing = dict()
        for key in review_data:
            subthing[key]= review_data[key]
        thing.append(subthing)
    return thing

In [ ]:
train = reading_in('../data/classification/music_reviews_train.json.gz')
dev = reading_in('../data/classification/music_reviews_dev.json.gz')
test = reading_in('../data/classification/music_reviews_test_masked.json.gz')

In [ ]:
train_x = [i['reviewText'] if "reviewText" in i.keys() else "" for i in train]
train_y =  [i['sentiment'] for i in train]

dev_x = [i['reviewText'] if "reviewText" in i.keys() else "" for i in dev]
dev_y = [i['sentiment'] for i in dev]

test_x = [i['reviewText'] if "reviewText" in i.keys() else "" for i in test]
test_y = [i['sentiment'] for i in test]

In [ ]:
def dumb(text):
    length = len(text)
    swap_p = random.randint(1,length-1)
    rem_p = random.randint(0,length-1)
    ##swap 2 random letters next to each other
    character = text[swap_p]
    text = text[:swap_p-1] + text[swap_p] + text[swap_p-1] + text[swap_p+1:]
    #remove a random character
    text =  text[:rem_p] +  text[rem_p+1:]
    return text


In [ ]:
def make_dumb(text):
    for i in range(math.ceil(len(text)/75)):
        text = dumb(text)
    return text

In [ ]:
make_dumb("The first time i heard")

'The ifrst timei heard'

### Correct format file thing

In [ ]:
list_of_json_dicts=[]

In [ ]:
dumb_text_x = [ make_dumb(i) for i in dev_x[:100]]
#dumb_text_x

In [ ]:
def create_json_format_dict(review,sentiment,category):
    return {"reviewText":review,"sentiment":sentiment,"category":category}
    

In [ ]:
def add_to_json_format(review,sentiment,category):
    while sentiment!= "negative" and sentiment!="positive":
        print(sentiment,"is an incorrect ")
        sentiment=input("sentiment must be negative or positive, enter sentiment again: ")
    list_of_json_dicts.append({"reviewText":review,"sentiment":sentiment,"category":category})

In [ ]:
#add_to_json_format("hello","negatv","hello")

In [ ]:
#handmade read in
from csv import reader
with open('../data/difficult_cases.csv', 'r', encoding = 'UTF-8') as f:
    csv_reader = reader(f)
    list_of_reviews = list(csv_reader)[1:]#split off header
print(list_of_reviews)

[["The first time I heard their album I really didn't like it it sounded awful. But then I heard it again in my headphones and I LOVE it now.", 'positive', 'Sentiment over time '], ['I heard the album for the first time a couple weeks ago and really liked it but then i heard it every day on my way to work and now i basically hate it.\xa0', 'negative', 'Sentiment over time '], ['My boyfriend introduced me to Arctic Monkeys and we loved hearing them together. Then I found the bastard cheating on me WHILE hearing Arctic Monkeys so now their music makes me cry like a baby and I hate it.\xa0', 'negative', 'Sentiment over time '], ['Had a perfect party hearing Avicii the whole evening. It was the best. I had too much tequila and got quite sick and now levels remind me of sitting in front of a toilet puking. I don’t wanna hear Avicii anymore because of that', 'negative', 'Sentiment over time '], ['I started hating Justin Bieber because everyone loved him. Now everyone hates him so I love him 

In [ ]:
#sort out \xa0
list_of_reviews = [[j.replace(u'\xa0', u' ').replace(u"2026", u"!" ).replace(u"\u2026", u"!" ) for j in i] for i in list_of_reviews]
print(list_of_reviews)

[["The first time I heard their album I really didn't like it it sounded awful. But then I heard it again in my headphones and I LOVE it now.", 'positive', 'Sentiment over time '], ['I heard the album for the first time a couple weeks ago and really liked it but then i heard it every day on my way to work and now i basically hate it. ', 'negative', 'Sentiment over time '], ['My boyfriend introduced me to Arctic Monkeys and we loved hearing them together. Then I found the bastard cheating on me WHILE hearing Arctic Monkeys so now their music makes me cry like a baby and I hate it. ', 'negative', 'Sentiment over time '], ['Had a perfect party hearing Avicii the whole evening. It was the best. I had too much tequila and got quite sick and now levels remind me of sitting in front of a toilet puking. I don’t wanna hear Avicii anymore because of that', 'negative', 'Sentiment over time '], ['I started hating Justin Bieber because everyone loved him. Now everyone hates him so I love him and hi

In [ ]:
#adding spelling mistakes
for x,i in enumerate(dumb_text_x):
    list_of_json_dicts.append(create_json_format_dict(i,dev_y[x],"spelling"))

In [ ]:
#handmade add to json list of dicts
for i in list_of_reviews:
    add_to_json_format(i[0],i[1],i[2])

negative  is an incorrect 
positive  is an incorrect 


In [ ]:
len(list_of_json_dicts)

166

In [ ]:
list_of_json_dicts_final=list_of_json_dicts

In [ ]:
len(list_of_json_dicts_final)

166

In [ ]:
#to document
test_json=[json.dumps(i)+"\n" for i in list_of_json_dicts_final]
with open ("../data/group6.json","w", encoding='utf-8') as file:
    file.writelines(test_json)

In [ ]:
#checking with the code given
import json
inputPath = '../data/group6.json'

for lineIdx, line in enumerate(open(inputPath)):
    try:
        data = json.loads(line)
    except ValueError as e:
        print('error, instance ' + str(lineIdx+1) + ' is not in valid json format')
        continue
    if 'reviewText' not in data:
        print("error, instance " + str(lineIdx+1) + ' does not contain key "reviewText"')
        continue
    if 'sentiment' not in data:
        print("error, instance " + str(lineIdx+1) + ' does not contain key "sentiment"')
        continue
    if data['sentiment'] not in ['positive', 'negative']:
        print("error, instance " + str(lineIdx+1) + ': sentiment is not positive/negative')
        continue
        
if lineIdx+1 < 100:
    print('Too little instances(' + str(lineIdx) + '), please generate more')
if lineIdx+1 > 1000:
    print('Too many instances(' + str(lineIdx) + '), please generate more')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b88dfe01-c7e1-473c-bcfd-798313fc6522' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>